In [1]:
from darkflow.net.build import TFNet
import cv2
from sort.sort import *

In [7]:
build_path = '../../utils/test_build'
weights = os.path.join(build_path, 'bin', 'yolo.weights')
model = os.path.join(build_path, 'cfg', 'yolo-people.cfg')
cfg = os.path.join(build_path, 'cfg')
labels = os.path.join(build_path, 'labels.txt')

In [8]:
options=dict()
#options['gpu'] = 0.5
options['model'] = model
options['load'] = -1
options['config'] = cfg
options['labels'] = labels
options['train'] = False
#options['dataset'] = images_dir
#options['val_dataset'] = images_dir
#options['annotation'] = train_annotations_dir
#options['val_annotation'] = val_annotations_dir
#options['epoch'] = 10
options['gpu'] = 0.8
#options["batch"] = 2
options["subdivisions"] = 1
options['backup'] = 'D:/data/models/tpe_retrained_all_obj/full_data/ckpt'
options['summary'] = None #'D:/data/models/tpe_retrain_nearbound/summary'
#options['save'] = 100
options['threshold'] = 0.5
#options['lr'] = 1e-4

In [9]:
tf_detector = TFNet(options)

Parsing ../../utils/test_build\cfg\yolo-people.cfg
Loading None ...
Finished in 0.0009970664978027344s

Building net ...
Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------
       |        | input                            | (?, 608, 608, 3)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 608, 608, 32)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 304, 304, 32)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 304, 304, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 152, 152, 64)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Init  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 152, 152, 64)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 76, 76, 128)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 76, 76, 256)
 Init  |  Yep!  | conv 1x1p0_1  +bnor

Loading from D:/data/models/tpe_retrained_all_obj/full_data/ckpt\yolo-people-10600
INFO:tensorflow:Restoring parameters from D:/data/models/tpe_retrained_all_obj/full_data/ckpt\yolo-people-10600
Finished in 14.138180494308472s



In [10]:
import numpy as np
import os

In [11]:
def video_proc(video_file_path, saving_path, tf_detector, max_imgs_per_person=50, delay=1):
    for file in os.listdir(video_file_path):
        print('processing file: {}'.format(file))

        tracker = Sort()

        camera = cv2.VideoCapture(os.path.join(video_file_path,file))
        while(1):
            ret, img_bgr = camera.read()

            if not ret:
                camera.release()
                break

            img = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
            pred_dict = tf_detector.return_predict(img)
            detections = get_boxes_from_dict(pred_dict)
            bboxes = tracker.update(detections)

            for box_ in bboxes:
                box = box_.astype(int)
                person_id = box[4]
                person_dir = os.path.join(saving_path, str(person_id))
                
                if not os.path.exists(person_dir):
                    os.mkdir(person_dir)
                
                n_img = len(os.listdir(person_dir))
                im_name = str(person_id) + '_' + str(n_img) + '.jpg'
                imCrop = img_bgr[box[1]:box[3], box[0]:box[2]]
                
                if n_img < max_imgs_per_person:
                    cv2.imwrite(os.path.join(person_dir,im_name), imCrop)
                
                cv2.rectangle(img_bgr, (box[0], box[1]), (box[2], box[3]), (255,0,0), 1)
                cv2.putText(img_bgr, 'id:' +str(box[4]),(box[0],box[1]),cv2.FONT_HERSHEY_SIMPLEX, 2, (0,255,0))
                
            cv2.imshow('ex', img_bgr)
            k = cv2.waitKey(delay)
            if k & 0xFF == ord('q'):
                camera.release()
                break

        if k & 0xFF == ord('q'):
            break
cv2.destroyAllWindows()
    

In [12]:
def get_boxes_from_dict(pred_dict):
    detections = None
    for pred in pred_dict:
        if pred['label'] == 'person':
            bbox = np.array(
                [int(pred['topleft']['x']),
                 pred['topleft']['y'],
                 pred['bottomright']['x'],
                 pred['bottomright']['y'],
                 pred['confidence']])
            if detections is None:
                detections = bbox[np.newaxis,:]
            else:
                detections = np.vstack([detections, bbox])
    if detections is None:
        detections = np.array([])
    return detections

In [13]:
saving_path = 'D:\\data\\pics\\persons_yolo_test'
video_file_path = 'D:\\data\\short\\cam_1'

In [14]:
video_proc(video_file_path, saving_path, tf_detector, max_imgs_per_person=50, delay=1)

processing file: 170814_170101 (1).avi
processing file: 170814_170201 (1).avi
processing file: 170814_170301 (1).avi
processing file: 170814_170401 (1).avi
processing file: 170814_170501 (1).avi
processing file: 170814_170601 (1).avi
processing file: 170814_170701 (1).avi
processing file: 170814_170801 (1).avi
processing file: 170814_170901 (1).avi
processing file: 170814_171001 (1).avi
processing file: 170814_171101 (1).avi
processing file: 170814_171201 (1).avi
processing file: 170814_171301 (1).avi
processing file: 170814_171401 (1).avi
processing file: 170814_171501 (1).avi
processing file: 170814_171601 (1).avi
processing file: 170814_171701 (1).avi
processing file: 170814_171801 (1).avi
processing file: 170814_171901 (1).avi
processing file: 170814_172001 (1).avi
processing file: 170814_172101 (1).avi
processing file: 170814_172201 (1).avi
processing file: 170814_172301.avi
processing file: 170814_172401.avi
processing file: 170814_172501 (1).avi
processing file: 170814_172601.av

In [15]:
cv2.destroyAllWindows()